# Laboratorio de Apache Spark en Kubernetes

##   

## Objetivos de Aprendizaje

En este laboratorio, usted:

- Creará un Pod de Kubernetes - un conjunto de contenedores que se ejecutan dentro de Kubernetes - aquí, que contiene Apache Spark que utilizamos para enviar trabajos a Kubernetes
- Enviará trabajos de Apache Spark a Kubernetes

## Resumen

Bienvenido al laboratorio sobre cómo enviar aplicaciones de Apache Spark a un clúster de Kubernetes. Este ejercicio es sencillo gracias al nuevo programador nativo de Kubernetes que se ha añadido a Spark recientemente.

Kubernetes es un orquestador de contenedores que permite programar millones de contenedores “docker” en enormes clústeres de computación que contienen miles de nodos de computación. Originalmente inventado y de código abierto por Google, Kubernetes se convirtió en el estándar de facto para el desarrollo y despliegue de aplicaciones nativas de la nube dentro y fuera de IBM. Con RedHat OpenShift, IBM es el líder en Kubernetes de nube híbrida y se encuentra entre las tres principales empresas que contribuyen a la base de código abierto de Kubernetes.

# Requisitos Previos

Nota: Si estás ejecutando este laboratorio dentro del entorno de Skillsnetwort Lab, todos los requisitos previos ya están instalados para ti.

Los únicos requisitos previos para este laboratorio son:

- Una instalación de _docker_ en funcionamiento
- Una instalación de _kubernetes_ en funcionamiento
- La herramienta de línea de comandos _git_

# Configuración

1. En el lado derecho de estas instrucciones verás el IDE Theia. Selecciona la pestaña _Lab_. En la barra de menú selecciona _Terminal\>Nuevo Terminal_.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/NewTerminal.png)

2. Por favor, ingresa el siguiente comando en el terminal para obtener el código más reciente.

```bash
git clone https://github.com/ibm-developer-skills-network/fgskh-new_horizons.git

```

3. Cambia el directorio al código descargado.

```bash
cd fgskh-new_horizons

```

4. Agrega un alias a `kubectl`. Esto te ayudará a escribir solo `k` en lugar de `kubectl`.

```bash
alias k='kubectl'

```

5. Guarda el espacio de nombres actual en una variable de entorno que se utilizará más tarde.

```bash
my_namespace=$(kubectl config view --minify -o jsonpath='{..namespace}')

```

# Desplegar el Pod de Apache Spark en Kubernetes

1. Instalar el POD de Apache Spark

```bash
k apply -f spark/pod_spark.yaml
```

2. Ahora es el momento de verificar el estado del Pod ejecutando el siguiente comando.

```bash
k get po
```

Si ves la siguiente salida, significa que el Pod aún no está disponible y necesitas esperar un poco.

```bash
NAME   READY   STATUS              RESTARTS   AGE
spark  0/2     ContainerCreating   0          29s
```

3. Espera unos segundos y emite el comando nuevamente después de un tiempo.

```bash
k get po
```

> Por favor, repite el paso 2 hasta que tengas un `STATUS` que refleje que está `Running`.

4. Deberías ver una salida como la que se muestra a continuación. El atributo `AGE` puede ser diferente, dependiendo de cuánto tiempo te tomó hacerlo funcionar.

```bash
NAME  READY   STATUS    RESTARTS   AGE
spark 2/2     Running   0          10m
```

> En caso de que veas el siguiente estado, necesitas eliminar el pod y comenzar de nuevo más tarde, ya que esto suele ocurrir cuando el registro de imágenes es poco fiable o está fuera de línea.

```bash
NAME   READY   STATUS              RESTARTS   AGE
spark  0/2     ImagePullBackOff    0          29s
```

5. Solo en este caso, por favor elimina el pod:

```bash
k delete po spark
```

Entonces comienza de nuevo:

```bash
k apply -f spark/pod_spark.yaml
```

De nuevo, verifica el estado regularmente:

```bash
k get po
```

Tenga en cuenta que este Pod se llama _spark_ y contiene dos contenedores _(2/2)_, ambos en estado _Running_. También tenga en cuenta que Kubernetes _RESTARTS_ automáticamente los pods fallidos; esto no ha sucedido aquí hasta ahora. Probablemente porque la _AGE_ de este pod es de solo 10 minutos.

# Enviar trabajos de Apache Spark a Kubernetes

Ahora es el momento de ejecutar un comando dentro del contenedor _spark_ de este Pod. Se le indica al comando _exec_ que proporcione acceso al contenedor llamado _spark_ (-c). Con _–_ ejecutamos un comando, en este ejemplo simplemente mostramos un mensaje.

```bash
k exec spark -c spark  -- echo "Hello from inside the container"
```

Acabas de ejecutar un comando en el contenedor _spark_ que reside en el pod _spark_ dentro de Kubernetes. Usaremos este contenedor para enviar aplicaciones Spark al clúster de Kubernetes. Este contenedor se basa en una imagen con la distribución de Apache Spark y el comando _kubectl_ preinstalado.

Si estás interesado, puedes echar un vistazo al [Dockerfile](https://github.com/romeokienzler/new_horizons/blob/main/spark/Dockerfile) para entender qué hay realmente dentro.

También puedes consultar el [pod.yaml](https://github.com/romeokienzler/new_horizons/blob/main/spark/pod_spark.yaml). Notarás que contiene dos contenedores. Uno es Apache Spark, y el otro proporciona un Proxy de Kubernetes - un contenedor llamado side car - que permite interactuar con el clúster de Kubernetes desde dentro de un Pod.

Dentro del contenedor puedes usar el comando _spark-submit_ que utiliza el nuevo programador nativo de Kubernetes que se ha añadido recientemente a Spark.

El siguiente comando envía la aplicación de muestra _SparkPi_ al clúster. SparkPi calcula Pi y cuantas más iteraciones ejecutes, más preciso se vuelve:

```bash
k exec spark -c spark -- ./bin/spark-submit \
--master k8s://http://127.0.0.1:8001 \
--deploy-mode cluster \
--name spark-pi \
--class org.apache.spark.examples.SparkPi \
--conf spark.executor.instances=1 \
--conf spark.kubernetes.container.image=romeokienzler/spark-py:3.1.2 \
--conf spark.kubernetes.executor.request.cores=0.2 \
--conf spark.kubernetes.executor.limit.cores=0.3 \
--conf spark.kubernetes.driver.request.cores=0.2 \
--conf spark.kubernetes.driver.limit.cores=0.3 \
--conf spark.driver.memory=512m \
--conf spark.kubernetes.namespace=${my_namespace} \
local:///opt/spark/examples/jars/spark-examples_2.12-3.1.2.jar \
10
```

Deberías ver una salida como la siguiente, por favor ignora las ADVERTENCIAS. A menos que no veas ERRORES, todo está bien:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/images/kube_lab_spark_submit_output.png)

# Entendiendo el comando spark-submit

Así que veamos qué está sucediendo aquí:

- _./bin/spark-submit_ es el comando para enviar aplicaciones a un clúster de Apache Spark
- _–master k8s://[http://127.0.0.1:8001](http://127.0.0.1:8001/)_ es la dirección del servidor API de Kubernetes - la forma en que _kubectl_ pero también el programador nativo de Kubernetes de Apache Spark interactúa con el clúster de Kubernetes
- _–name spark-pi_ proporciona un nombre para el trabajo y los Pods subsiguientes creados por el programador nativo de Kubernetes de Apache Spark se prefijan con ese nombre
- _–class org.apache.spark.examples.SparkPi_ proporciona el nombre canónico de la aplicación Spark que se va a ejecutar (nombre del paquete Java y nombre de la clase)
- _–conf spark.executor.instances=1_ le dice al programador nativo de Kubernetes de Apache Spark cuántos Pods debe crear para paralelizar la aplicación. Tenga en cuenta que en este clúster de desarrollo de un solo nodo, aumentar este número no tiene sentido (además de agregar sobrecarga por paralelización)
- _–conf spark.kubernetes.container.image=romeokienzler/spark-py:3.1.2_ le dice al programador nativo de Kubernetes de Apache Spark qué imagen de contenedor debe usar para crear los Pods del driver y del executor. Esta imagen se puede construir de forma personalizada utilizando los Dockerfiles proporcionados en _kubernetes/dockerfiles/spark/_ y _bin/docker-image-tool.sh_ en la distribución de Apache Spark
- _–conf spark.kubernetes.executor.limit.cores=0.3_ le dice al programador nativo de Kubernetes de Apache Spark que establezca el límite de núcleos de CPU para usar solo 0.3 núcleo por Pod executor
- _–conf spark.kubernetes.driver.limit.cores=0.3_ le dice al programador nativo de Kubernetes de Apache Spark que establezca el límite de núcleos de CPU para usar solo 0.3 núcleo para el Pod del driver
- _–conf spark.driver.memory=512m_ le dice al programador nativo de Kubernetes de Apache Spark que establezca el límite de memoria para usar solo 512MB para el Pod del driver
- _–conf spark.kubernetes.namespace=${my\_namespace}_ le dice al programador nativo de Kubernetes de Apache Spark que establezca el namespace a la variable de entorno _my\_namespace_ que configuramos antes.
- _local:///opt/spark/examples/jars/spark-examples\_2.12-3.1.2.jar_ indica el archivo _jar_ que contiene la aplicación. Tenga en cuenta que el prefijo _local://_ se refiere a una ruta dentro de las imágenes de contenedor proporcionadas por la opción _spark.kubernetes.container.image_. Dado que estamos utilizando un _jar_ proporcionado por la distribución de Apache Spark, esto no es un problema; de lo contrario, la opción _spark.kubernetes.file.upload.path_ debe configurarse y debe configurarse un subsistema de almacenamiento apropiado, como se describe en la [documentación](https://spark.apache.org/docs/latest/running-on-kubernetes.html#running-spark-on-kubernetes)
- _10_ le dice a la aplicación que se ejecute durante _10_ iteraciones, luego muestre el valor calculado de _Pi_

Por favor, consulte la [documentación](https://spark.apache.org/docs/latest/running-on-kubernetes.html#configuration) para obtener una lista completa de los parámetros disponibles.

# Monitorear la aplicación Spark en una terminal paralela

Una vez que se ejecute este comando, puedes _abrir una segunda ventana de terminal_ dentro de Theia y emitir el siguiente comando:

> **Nota:** Para ver al menos un ejecutor, ejecuta el comando mencionado a continuación mientras la otra terminal aún está ejecutando el comando spark-submit.

```bash
kubectl get po
```

Esto te mostrará los Pods adicionales que están siendo creados por el programador nativo de Kubernetes de Apache Spark: un controlador y al menos un ejecutor. Ten en cuenta que con solo un ejecutor, el controlador puede ejecutar el ejecutor dentro de su propio pod. Aquí hay un ejemplo cuando se utiliza un ejecutor que se ejecuta por separado del pod del controlador (los IDs exactos se reemplazan por X y Y para facilitar la lectura):

```bash
NAME              READY STATUS    RESTARTS AGE
spark             2/2   Running   0        28m
spark-pi-X-exec-1 1/1   Running   0        33s
spark-pi-X-driver 1/1   Running   0        44s
spark-pi-Y-driver 0/1   Completed 0        12m
```

Puedes ver que el Pod _spark-pi-Y-driver_ está en estado _Completed_, de una ejecución de un solo ejecutor hace doce minutos y que hay un controlador y tres ejecutores actualmente en ejecución para el trabajo _spark-pi-X- .._.

Para verificar el tiempo transcurrido del trabajo, simplemente ejecuta (necesitas reemplazar el nombre del Pod, por supuesto, con el que está en tu sistema):

<span data-darkreader-inline-color="" style="scrollbar-color: rgb(69, 74, 77) rgb(32, 35, 36); padding: 0px; margin: 0px; box-sizing: border-box; color: red; --darkreader-inline-color: #ff1a1a;">Por favor, asegúrate de ejecutar el siguiente código en la nueva ventana de terminal que te permite ejecutar comandos dentro del controlador de Spark que se está ejecutando en un POD.</span>

> **Nota:** Reemplaza el ID en el Spark-pi-ID-driver con el que has creado tú. Por ejemplo: si tu pod es spark-pi-6f62d17a800beb3e-driver, entonces reemplaza ID con 6f62d17a800beb3e.

```bash
kubectl logs spark-pi-6f62d17a800beb3e-driver |grep "Job 0 finished:"
```

Deberías obtener algo como:

```bash
Job 0 finished: reduce at SparkPi.scala:38, took 8.446024 s
```

Si estás interesado en saber qué valor para _Pi_ generó la aplicación, simplemente emite:

> **Nota:** Reemplaza el ID en el Spark-pi-ID-driver con el que tú creaste. Por ejemplo: si tu pod es spark-pi-6f62d17a800beb3e-driver, entonces reemplaza el ID con 6f62d17a800beb3e.

```bash
kubectl logs spark-pi-6f62d17a800beb3e-driver |grep "Pi is roughly "
```

Y verás algo como:

```bash
Pi is roughly 3.1416551416551415
```

# Experimenta tú mismo

Ahora puedes experimentar con valores para _spark.executor.instances_, _spark.kubernetes.executor.limit.cores=0.5_ (0.1 también es un número válido) y el número de iteraciones y ver cómo afecta el tiempo de ejecución y la precisión del resultado. Solo asegúrate de no exceder el límite de cuota de recursos de SkillsNetwork. Observa `Kubectl logs [driver pod]` para verificar los registros por si se excede la cuota.

Esto concluye este laboratorio.

# Resumen

En este laboratorio has aprendido a crear un POD cliente de Apache Spark dentro del clúster de Kubernetes para enviar trabajos. Luego, has utilizado el comando spark-submit para crear un trabajo que se ejecuta dentro de este clúster de Kubernetes. Ahora eres capaz de escalar tus trabajos de Apache Spark en cualquier clúster de Kubernetes que se ejecute en la nube o en tu centro de datos a miles de nodos, CPUs y GB de memoria principal.